# Checking the monitoring component

In [1]:
import time
import datetime
import uuid
import random

from dotenv import load_dotenv
from IPython.display import clear_output, display, HTML

from common.tools.timestamp.Converters import timestamp_utcnow_to_float, timestamp_float_to_string
from common.tools.grpc.Tools import grpc_message_to_json_string
from common.proto.kpi_sample_types_pb2 import KpiSampleType
from common.proto.monitoring_pb2 import KpiDescriptor, KpiId, KpiQuery, Kpi
from monitoring.client.MonitoringClient import MonitoringClient

In [2]:
load_dotenv()

monitoring_client = MonitoringClient()
uuid.uuid4().hex

'0abfb00117d4461b9fa5085bee4be58f'

In [3]:
kpi_description: KpiDescriptor = KpiDescriptor()
kpi_description.kpi_description = "Security status of service {}".format(uuid.uuid4().hex)
kpi_description.service_id.service_uuid.uuid = "608df176-90b8-5950-b50d-1810c6eaaa5d"
kpi_description.kpi_sample_type = KpiSampleType.KPISAMPLETYPE_UNKNOWN
new_kpi = monitoring_client.SetKpi(kpi_description)
print("Created KPI {}: ".format(grpc_message_to_json_string(new_kpi)))

Created KPI {"kpi_id": {"uuid": "1"}}: 


In [4]:
kpi_id = input("What is the KPI ID?")
query = KpiQuery()
query.kpi_ids.append(KpiId(**{"kpi_id": {"uuid": kpi_id}}))
query.last_n_samples = 10

while True:

    kpi = Kpi()
    kpi.kpi_id.kpi_id.uuid = new_kpi.kpi_id.uuid
    kpi.timestamp.timestamp = timestamp_utcnow_to_float()
    kpi.kpi_value.int32Val = random.randint(10, 4000)
    # monitoring_client.IncludeKpi(kpi)

    response = monitoring_client.QueryKpiData(query)
    # print(response)
    table = f"""<table>
    <thead>
        <tr><th colspan=3>{datetime.datetime.now()}</th></tr>
        <tr><th>KPI ID</th><th>Timestamp</th><th>Value</th></tr>
    <thead>
    <tbody>"""
    for kpi in response.raw_kpi_lists:
        cur_kpi_id = kpi.kpi_id.kpi_id.uuid
        for i, raw_kpi in enumerate(kpi.raw_kpis):
            # print(cur_kpi_id, raw_kpi.timestamp.timestamp, raw_kpi.kpi_value)
            table += "<tr><td>{} - {}</td><td>{}</td><td>{}</td></tr>".format(
                i, cur_kpi_id, timestamp_float_to_string(raw_kpi.timestamp.timestamp), raw_kpi.kpi_value
            )
    table += "</tbody></table>"
    display(HTML(table))
    time.sleep(5)
    clear_output(wait=True)

KeyboardInterrupt: 